In [7]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from geoloc_functions import find_location,distance_to_centre, direction_from_centre
from openlocationcode import openlocationcode as olc
from pluscode_functions import convert_dic, convert_pluscode
from iris_functions import create_iris_polygon_dict, find_iris

In [2]:
# Read recipe inputs
new_inputs = dataiku.Dataset("new_inputs")
new_inputs_df = new_inputs.get_dataframe()

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [3]:
new_inputs_df

,adresse,code_postal,surface_m2_not_null,date_mutation_year,nombre_pieces_principales
0,13 rue Saulnier,75009,75,2022,3


In [5]:
new_inputs_df['latitude'] = find_location(new_inputs_df['adresse'])[0]
new_inputs_df['longitude'] = find_location(new_inputs_df['adresse'])[1]

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [6]:
new_inputs_df

,adresse,code_postal,surface_m2_not_null,date_mutation_year,nombre_pieces_principales,latitude,longitude
0,13 rue Saulnier,75009,75,2022,3,48.874973,2.344335


In [8]:
new_inputs_df['distance_to_centre']= distance_to_centre(new_inputs_df['latitude'],new_inputs_df['longitude'])

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [9]:
new_inputs_df['direction_from_centre'] = np.vectorize(direction_from_centre)(new_inputs_df['longitude'],new_inputs_df['latitude'])

In [10]:
new_inputs_df

,adresse,code_postal,surface_m2_not_null,date_mutation_year,nombre_pieces_principales,latitude,longitude,distance_to_centre,direction_from_centre
0,13 rue Saulnier,75009,75,2022,3,48.874973,2.344335,2.059611,N


In [14]:
new_inputs_df['pluscode_16_first'] = np.vectorize(olc.encode)(new_inputs_df['latitude'],new_inputs_df['longitude'],17)

In [15]:
new_inputs_df['pluscode_10'] = [code[:11] for code in new_inputs_df['pluscode_16_first'].to_list()]

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [16]:
new_inputs_df['v_coordinate'],new_inputs_df['h_coordinate'],new_inputs_df['reduced_v_coordinate'],new_inputs_df['reduced_h_coordinate'],new_inputs_df['reduced_coordinates_couple'] = convert_pluscode(new_inputs_df['pluscode_10'])

In [17]:
iris_75 = dataiku.Dataset("iris_75")
iris_75_df = iris_75.get_dataframe()

In [18]:
iris_coords_codes = create_iris_polygon_dict(iris_75_df['CODE_IRIS'],iris_75_df['Geo Shape'])

In [19]:
iris_coords_codes

{751010204: <shapely.geometry.polygon.Polygon at 0x142464f10>,
 751010103: <shapely.geometry.polygon.Polygon at 0x1424d66d0>,
 751010303: <shapely.geometry.polygon.Polygon at 0x142e256d0>,
 751010105: <shapely.geometry.polygon.Polygon at 0x142e257d0>,
 751010199: <shapely.geometry.polygon.Polygon at 0x142e25810>,
 751010203: <shapely.geometry.polygon.Polygon at 0x142e255d0>,
 751010104: <shapely.geometry.polygon.Polygon at 0x142e258d0>,
 751010205: <shapely.geometry.polygon.Polygon at 0x142e25910>,
 751010101: <shapely.geometry.polygon.Polygon at 0x142e25950>,
 751010402: <shapely.geometry.polygon.Polygon at 0x142e25850>,
 751010401: <shapely.geometry.polygon.Polygon at 0x142e25890>,
 751010202: <shapely.geometry.polygon.Polygon at 0x142e259d0>,
 751010206: <shapely.geometry.polygon.Polygon at 0x142e25a10>,
 751010301: <shapely.geometry.polygon.Polygon at 0x142e25a50>,
 751010302: <shapely.geometry.polygon.Polygon at 0x142e25a90>,
 751010201: <shapely.geometry.polygon.Polygon at 0x142e

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

inputs_located_df = new_inputs_df # For this sample code, simply copy input to output


# Write recipe outputs
inputs_located = dataiku.Dataset("inputs_located")
inputs_located.write_with_schema(inputs_located_df)